# setup

In [1]:
!nvidia-smi

Mon Nov 25 12:29:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:85:00.0 Off |                    0 |
| N/A   35C    P0             44W /  300W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install langchain langchain-huggingface bitsandbytes datasets matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import getpass
import os
from datasets import load_dataset

In [4]:
api_env_key = "HUGGINGFACEHUB_API_TOKEN"
if os.environ.get(api_env_key) is None:
    os.environ[api_env_key] = getpass.getpass(
        "Enter your Hugging Face API key: "
    )

Enter your Hugging Face API key:  ········


In [5]:
# Load shared utility methods
%run SharedUtils.ipynb

In [6]:
# Load LLAMA
%run LoadLLAMA.ipynb

2024-11-25 12:29:58.351669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 12:29:58.376878: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 12:29:58.384554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 12:29:58.405174: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-25 12:30:00.497029: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

128009
<class 'int'>


In [7]:
dataset = load_dataset("dair-ai/emotion", "split")

train_set = dataset["train"]
test_set = dataset["test"]

print(train_set[:5])
print(test_set[:5])

{'text': ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy'], 'label': [0, 0, 3, 2, 3]}
{'text': ['im feeling rather rotten so im not very ambitious right now', 'im updating my blog because i feel shitty', 'i never make her separate from me because i don t ever want her to feel like i m ashamed with her', 'i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived', 'i was feeling a little vain when i did this one'], 'label': [0, 0, 0, 1, 0]}


In [8]:
label_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}
label_values = list(label_map.values())

train_set = train_set.map(map_labels)
test_set = test_set.map(map_labels)

X_test = test_set["text"]
y_test = test_set["label_human"]

print(train_set[:5])
print(test_set[:5])

{'text': ['i didnt feel humiliated', 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'im grabbing a minute to post i feel greedy wrong', 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property', 'i am feeling grouchy'], 'label': [0, 0, 3, 2, 3], 'label_human': ['sadness', 'sadness', 'anger', 'love', 'anger']}
{'text': ['im feeling rather rotten so im not very ambitious right now', 'im updating my blog because i feel shitty', 'i never make her separate from me because i don t ever want her to feel like i m ashamed with her', 'i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived', 'i was feeling a little vain when i did this one'], 'label': [0, 0, 0, 1, 0], 'label_human': ['sadness', 'sadness', 'sadness', 'joy', 'sadness']}


In [9]:
def get_reasoning(label):
    reasoning_map = {
        0: "This sentence conveys a sense of loss, disappointment, or unhappiness, often associated with words or phrases indicating regret or sorrow.",
        1: "This sentence includes positive expressions of happiness, satisfaction, or excitement, often using words that suggest joy or celebration.",
        2: "This sentence expresses affection, admiration, or strong emotional connections, often reflecting bonds of love or devotion.",
        3: "This sentence contains language that conveys frustration, irritation, or hostility, often marked by aggressive or critical tones.",
        4: "This sentence reflects apprehension, worry, or a sense of threat, often using language that conveys uncertainty or fear.",
        5: "This sentence suggests surprise or astonishment, often with words or phrases that indicate unexpected outcomes or disbelief."
    }
    return reasoning_map.get(label)

In [10]:
few_shot_examples, examples_text, examples_text_cot = prepare_examples(train_set, key="text", n_per_label=1)

print(few_shot_examples)
print("---")
print(examples_text)
print("---")
print(examples_text_cot)

[{'text': 'im feel alone and i dont know how to cope', 'label': 0, 'label_human': 'sadness'}, {'text': 'i feel really angry sometimes because for the love of god havent we been through enough', 'label': 3, 'label_human': 'anger'}, {'text': 'ive been feeling a bit nostalgic ever since i went through a box of my dads old pictures for a post i did for a href http mysalvagedtreasures', 'label': 2, 'label_human': 'love'}, {'text': 'i had a hard time focusing on my life and walked around feeling dazed and confused', 'label': 5, 'label_human': 'surprise'}, {'text': 'i have just had such a crappy week that i am still feeling all agitated and like the day wasn t what i wanted', 'label': 4, 'label_human': 'fear'}, {'text': 'id probably go with none on and hope that my date admires a confident girl who feels fine without makeup', 'label': 1, 'label_human': 'joy'}]
---
Sentence: im feel alone and i dont know how to cope
Sentiment: sadness
Sentence: i feel really angry sometimes because for the lov

# Zero-Shot Prompting Preview

In [11]:
ai_msg = zero_shot("The food was absolutely amazing and delightful!")
print(ai_msg)

ai_msg = zero_shot("The experience was the worst I've ever had.")
print(ai_msg)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Your goal is to read a sentence and classify its sentiment into one of the following categories: sadness, joy, love, anger, fear, surprise.

Now classify the following sentence. The output MUST follow this format:
Sentiment: [Classification]<|eot_id|><|start_header_id|>user<|end_header_id|>

The food was absolutely amazing and delightful!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Sentiment: joy
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Your goal is to read a sentence and classify its sentiment into one of the following categories: sadness, joy, love, anger, fear, surprise.

Now classify the following sentence. The output MUST follow this format:
Sentiment: [Classification]<|eot_id|><|start_header_id|>user<|end_header_id|>

The experience was the worst I've ever had.<|eo

# Few-Shot Prompting Preview

In [12]:
ai_msg = few_shot("The food was absolutely amazing and delightful!", examples_text)
print(ai_msg)

ai_msg = few_shot("The experience was the worst I've ever had.", examples_text)
print(ai_msg)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Your goal is to read a sentence and classify its sentiment into one of the following categories: sadness, joy, love, anger, fear, surprise.

Here are some examples:
Sentence: im feel alone and i dont know how to cope
Sentiment: sadness
Sentence: i feel really angry sometimes because for the love of god havent we been through enough
Sentiment: anger
Sentence: ive been feeling a bit nostalgic ever since i went through a box of my dads old pictures for a post i did for a href http mysalvagedtreasures
Sentiment: love
Sentence: i had a hard time focusing on my life and walked around feeling dazed and confused
Sentiment: surprise
Sentence: i have just had such a crappy week that i am still feeling all agitated and like the day wasn t what i wanted
Sentiment: fear
Sentence: id probably go with none on and hope that my date admires a confident girl who feels fine without 

# Few-Shot with Chain-of-Thought Preview

In [13]:
ai_msg = few_shot("The food was absolutely amazing and delightful!", examples_text_cot)
print(ai_msg)

ai_msg = few_shot("The experience was the worst I've ever had.", examples_text_cot)
print(ai_msg)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Your goal is to read a sentence and classify its sentiment into one of the following categories: sadness, joy, love, anger, fear, surprise.

Here are some examples:
Sentence: im feel alone and i dont know how to cope
Reasoning: This sentence conveys a sense of loss, disappointment, or unhappiness, often associated with words or phrases indicating regret or sorrow.
Sentiment: sadness
Sentence: i feel really angry sometimes because for the love of god havent we been through enough
Reasoning: This sentence contains language that conveys frustration, irritation, or hostility, often marked by aggressive or critical tones.
Sentiment: anger
Sentence: ive been feeling a bit nostalgic ever since i went through a box of my dads old pictures for a post i did for a href http mysalvagedtreasures
Reasoning: This sentence expresses affection, admiration, or strong emotional conn

# Zero-Shot Evaluation

In [14]:
# sampleset = test_set.select(range(100))

In [ ]:
zero_shot_results = evaluate_model(X_test, y_test, zero_shot, None)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
create_reports(y_test, zero_shot_results)

# Few-Shot Evaluation

In [ ]:
few_shot_results = evaluate_model(X_test, y_test, few_shot, examples_text)

In [ ]:
create_reports(y_test, few_shot_results)

# Few-Shot Chain-of-Thought Evaluation

In [ ]:
few_shot_cot_results = evaluate_model(X_test, y_test, few_shot, examples_text_cot)

In [ ]:
create_reports(y_test, few_shot_cot_results)